<a href="https://colab.research.google.com/github/OmarMachuca851/Task/blob/main/sec2sec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# sec2sec

## Ejecución de traducción automática y lectura de código

In [ ]:
import numpy as np
import keras
import os
from pathlib import Path

"""
## Download the data
"""

# fpath = keras.utils.get_file(origin="http://www.manythings.org/anki/fra-eng.zip")
# dirpath = Path(fpath).parent.absolute()
# os.system(f"unzip -q {fpath} -d {dirpath}")

"""
## Configuration
"""

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = os.path.join("./data/fra-eng", "fra.txt")

"""
## Prepare the data
"""

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype="float32",
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

"""
## Build the model
"""

# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

"""
## Train the model
"""

model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s_model.keras")

"""
## Run inference (sampling)

1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state
and a "start of sequence" token as target.
Output will be the next target token.
3. Repeat with the current target token and current states
"""

# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s_model.keras")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence


"""
You can now generate decoded sentences as such:
"""

for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

Using TensorFlow backend.


Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 91
Max sequence length for inputs: 14
Max sequence length for outputs: 59
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2025-09-28 21:23:24.250434: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2025-09-28 21:23:24.331059: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-09-28 21:23:24.335332: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: NVIDIA GeForce RTX 2060 major: 7 minor: 5 memoryClockRate(GHz): 1.65
pciBusID: 0000:01:00.0
2025-09-28 21:23:24.335485: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2025-09-28 21:23:24.336200: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2025-09-28 21:23:24.336888: I tensorf

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
  64/8000 [..............................] - ETA: 49s - loss: 4.5092 - accuracy: 0.0045

2025-09-28 21:23:25.153221: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0


8000/8000 [==============================] - 3s 392us/step - loss: 1.1304 - accuracy: 0.7385 - val_loss: 1.0124 - val_accuracy: 0.7209
Epoch 2/100
8000/8000 [==============================] - 3s 326us/step - loss: 0.8223 - accuracy: 0.7798 - val_loss: 0.8074 - val_accuracy: 0.7703
Epoch 3/100
8000/8000 [==============================] - 3s 324us/step - loss: 0.6515 - accuracy: 0.8169 - val_loss: 0.6833 - val_accuracy: 0.8031
Epoch 4/100
8000/8000 [==============================] - 3s 330us/step - loss: 0.5644 - accuracy: 0.8359 - val_loss: 0.6167 - val_accuracy: 0.8176
Epoch 5/100
8000/8000 [==============================] - 3s 318us/step - loss: 0.5151 - accuracy: 0.8494 - val_loss: 0.5830 - val_accuracy: 0.8276
Epoch 6/100
8000/8000 [==============================] - 2s 307us/step - loss: 0.4791 - accuracy: 0.8601 - val_loss: 0.5489 - val_accuracy: 0.8395
Epoch 7/100
8000/8000 [==============================] - 2s 307us/step - loss: 0.4517 - accuracy: 0.8673 - val_loss: 0.5309 - val_

### Lectura del código:
en mi caso utiliza datos locales (fra.txt)
- linea 18-23: configuración de variables globales
- linea 30-91: preparación de los datos (decodificación y tokenización)
- linea 98-118: construcción del modelo
- linea 124-133: entrenamiento del modelo
- linea 135: guardado del modelo
- linea 139: cargado del modelo
- linea 151-169: obtención de los datos códificados y decodificación de los datos
- linea 173-174: detokenización (reversión de codificación a caracteres para ser posible una lectura)
- linea 177-211: funcion de decodificación de secuencia
- linea 218-225: parte del código que permite comprobar el funcionamiento de las predicciones

## Ejecución de un modelo preentrenado para subtítulos de imágenes

In [ ]:
!python pytorchtutorial/tutorials/03-advanced/image_captioning/sample.py --image="pytorchtutorial/tutorials/03-advanced/image_captioning/png/example.png"

<start> a group of giraffes standing next to each other . <end>


c:\Users\OMC\Documents\DProClass\python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\OMC\Documents\DProClass\python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Investigar qué hacer si quieres ejecutarlo con Keras

### Consideraciones importantes:
- Compatibilidad de arquitecturas: Las arquitecturas deben ser equivalentes

- Orden de dimensiones: PyTorch usa (C, H, W), Keras usa (H, W, C)

- Nombres de capas: Pueden diferir entre frameworks

- Operaciones específicas: Algunas operaciones pueden no ser directamente transferibles

In [ ]:
import torch
import numpy as np

def load_pytorch_weights(keras_model, pytorch_weights_path):
    """
    Carga pesos desde un modelo PyTorch .pkl a un modelo Keras
    """
    # Cargar pesos de PyTorch
    pytorch_weights = torch.load(pytorch_weights_path, map_location='cuda:0')

    if 'state_dict' in pytorch_weights:
        pytorch_weights = pytorch_weights['state_dict']

    # Mapeo de nombres de capas
    keras_weights = []

    # Extraer pesos en el orden correcto para Keras
    for name, param in pytorch_weights.items():
        if 'resnet' in name and 'weight' in name and param.ndim == 4:
            # Pesos convolucionales: (out_c, in_c, h, w) -> (h, w, in_c, out_c)
            weight = param.numpy().transpose(2, 3, 1, 0)
            keras_weights.append(weight)
        elif 'linear.weight' in name:
            # Capa densa: (out_features, in_features) -> (in_features, out_features)
            weight = param.numpy().transpose(1, 0)
            keras_weights.append(weight)
        elif 'linear.bias' in name or 'bn' in name:
            # Sesgos y parámetros de BatchNorm
            keras_weights.append(param.numpy())

    # Establecer pesos en el modelo Keras
    keras_model.set_weights(keras_weights)

    return keras_model

# Uso con carga de pesos
# encoder = create_encoder_cnn(embed_size=256)
# encoder = load_pytorch_weights(encoder, 'pytorch_weights.pkl')

## Lectura y reescritura de código

### encoder

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.layers import Dense, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model

class EncoderCNN(keras.Model):
    def __init__(self, embed_size):
        super(EncoderCNN, self).__init__()

        # Cargar ResNet-152 preentrenado
        self.resnet_base = ResNet152(
            weights='imagenet',
            include_top=False,
            input_shape=(224, 224, 3)
        )

        # Congelar pesos de ResNet
        self.resnet_base.trainable = False

        # Capas adicionales
        self.global_pool = GlobalAveragePooling2D()
        self.linear = Dense(embed_size)
        self.bn = BatchNormalization(momentum=0.01)

    def forward(self, images, training=False):
        # Preprocesar imágenes para ResNet
        x = preprocess_input(images)

        # Pasar por ResNet (siempre en modo inference)
        x = self.resnet_base(x, training=False)

        # Global Average Pooling
        x = self.global_pool(x)

        # Capas fully connected
        x = self.linear(x)
        x = self.bn(x, training=training)

        return x

234698864/234698864 ━━━━━━━━━━━━━━━━━━━━ 68s 0us/step


### decoder

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense, Embedding, Input
import numpy as np

class DecoderRNN(keras.Model):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers, max_seq_length=20):
        """Set the hyper-parameters and build the layers."""
        super(DecoderRNN, self).__init__()
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.num_layers = num_layers
        self.max_seq_length = max_seq_length

        # Capas del modelo
        self.embed = Embedding(vocab_size, embed_size)
        self.lstm = LSTM(
                        hidden_size,
                        return_sequences=True,
                        dropout=0.1,
                        recurrent_dropout=0.1,
                        kernel_initializer='glorot_uniform',
                        recurrent_initializer='orthogonal'
                    )
  # Opcional: dropout para regularización
        self.linear = Dense(vocab_size)

    def call(self, inputs, training=False):
        """
        Forward pass para entrenamiento
        inputs: tuple (features, captions, lengths)
        """
        features, captions, lengths = inputs

        # Embedding de los captions
        embeddings = self.embed(captions)  # (batch_size, seq_len, embed_size)

        # Concatenar features con embeddings (como en PyTorch)
        features_expanded = tf.expand_dims(features, 1)  # (batch_size, 1, embed_size)
        embeddings = tf.concat([features_expanded, embeddings], axis=1)

        # Para manejar sequences padding en Keras, usamos mask_zero en Embedding
        # y el LSTM manejará automáticamente las máscaras

        # LSTM
        lstm_output, state_h, state_c = self.lstm(embeddings, training=training)

        # Capa lineal de salida
        outputs = self.linear(lstm_output)

        return outputs

    def forward(self, data):
        """
        Personalizar el paso de entrenamiento si es necesario
        """
        features, captions, lengths = data
        targets = captions[:, 1:]  # Shift para teacher forcing

        with tf.GradientTape() as tape:
            # Forward pass
            predictions = self((features, captions, lengths), training=True)

            # Calcular pérdida (usando solo las posiciones válidas)
            mask = tf.sequence_mask(lengths, maxlen=tf.shape(predictions)[1])
            loss = self.compiled_loss(targets, predictions, sample_weight=mask)

        # Backward pass
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        return {'loss': loss}

    def sample(self, features, states=None):
        """Generate captions for given image features using greedy search."""
        sampled_ids = []

        # Estado inicial
        if states is None:
            states = [tf.zeros((features.shape[0], self.hidden_size))
                     for _ in range(2 * self.num_layers)]

        # Input inicial son las features
        inputs = tf.expand_dims(features, 1)  # (batch_size, 1, embed_size)

        for i in range(self.max_seq_length):
            # LSTM forward
            lstm_output, state_h, state_c = self.lstm(inputs, initial_state=states, training=False)
            states = [state_h, state_c]

            # Capa lineal
            outputs = self.linear(tf.squeeze(lstm_output, 1))  # (batch_size, vocab_size)

            # Predicción greedy
            predicted = tf.argmax(outputs, axis=1)  # (batch_size,)
            sampled_ids.append(predicted)

            # Siguiente input
            inputs = self.embed(predicted)  # (batch_size, embed_size)
            inputs = tf.expand_dims(inputs, 1)  # (batch_size, 1, embed_size)

        sampled_ids = tf.stack(sampled_ids, axis=1)  # (batch_size, max_seq_length)
        return sampled_ids


## Cargar los pesos

In [ ]:
import pickle
from build_vocab import Vocabulary
with open('pytorchtutorial/tutorials/03-advanced/image_captioning/data/vocab.pkl', 'rb') as f:
        vocab = pickle.load(f)

encoder = EncoderCNN(embed_size=256)

decoder = DecoderRNN(
    embed_size=256,
    hidden_size=512,
    vocab_size=len(vocab),
    num_layers=1,
    max_seq_length=20
)

In [ ]:
load_pytorch_weights(encoder, "pytorchtutorial/tutorials/03-advanced/image_captioning/models/encoder-5-3000.pkl")
load_pytorch_weights(decoder, "pytorchtutorial/tutorials/03-advanced/image_captioning/models/decoder-5-3000.pkl")

## Investigación avanzada

### ¿Qué pasa si necesito traducir a otro idioma?

**1. Cambio de datos de entrenamiento**  
Reemplazarías el archivo fra-eng (francés-inglés) por un archivo jpn-eng (japonés-inglés)  
El formato sería el mismo: pares de oraciones paralelas, una en japonés y otra en inglés

**2. Preprocesamiento específico para japonés**  
Tokenización diferente: El japonés requiere tokenización especial (mecab, janome, etc.) ya que no tiene espacios entre palabras  
Caracteres especiales: El japonés usa múltiples sistemas de escritura (kanji, hiragana, katakana)  
Vocabulario más grande: Debido a los kanji, el vocabulario puede ser significativamente mayor  

**3. Modificaciones en el modelo**  
Tamaño de vocabulario: Aumentaría considerablemente para el lado japonés  
Dimensionalidad de embeddings: Podría necesitarse mayor dimensión para capturar la complejidad del japonés  
Longitud de secuencias: Las secuencias en japonés pueden ser más cortas en términos de tokens pero más complejas  

**4. Hiperparámetros ajustados**  
Mayor capacidad del modelo para manejar la complejidad lingüística  
Posiblemente más épocas de entrenamiento  
Regularización adicional para evitar sobreajuste  

**5. Evaluación diferente**  
Métricas de evaluación específicas para pares de idiomas distantes  
Consideración de diferencias estructurales entre japonés e inglés  


En esencia: El modelo arquitectónico se mantiene igual, pero todos los componentes relacionados con el procesamiento del lenguaje de entrada (tokenización, vocabulario, embeddings) cambian completamente para adaptarse al japonés.

### ¿Cuáles son algunos métodos avanzados de traducción automática?

Los grandes modelos de lenguaje (LLM, por sus siglas en inglés) han conquistado el mundo.
Algunos de estos modelos, como el GPT-4 de OpenAI y el PaLM2 de Google, se han entrenado con conjuntos de datos multilingües y, al menos en teoría, también deberían ser muy capaces de realizar tareas de traducción automática.  

Los grandes modelos de lenguaje suelen basarse también en estructuras de transformadores. Sin embargo, en comparación con los modelos de traducción automática neuronal (NMT, por sus siglas en inglés) del apartado anterior, se entrenan con volúmenes de texto mucho más extensos y contienen más parámetros del modelo. Los LLM contienen miles de millones de parámetros frente a los pocos cientos de millones de los modelos de NMT bilingües de una sola tarea. Esto hace que los modelos LLM sean más flexibles e «inteligentes» a la hora de interpretar las instrucciones o los «mensajes» del usuario. Esta nueva tecnología abre numerosas posibilidades en términos de adaptación de los modelos mediante el uso de datos empresariales. Dada la potencia de este planteamiento, prefiero hablar en términos de "personalización" en lugar de "adaptación".

### ¿Cómo generar imágenes a partir de texto?

**Arquitectura Típica de un Sistema Texto-Imagen**
- Encoder de Texto:  
Convierte el prompt en representación numérico
Modelos como CLIP, BERT, T5

- Space de Representación Alineado:  
Donde texto e imagen se "encuentran" conceptualmente  
Aprenden que "un gato negro" corresponde a ciertos patrones visuales  

- Generador de Imágenes:  
Transforma la representación alineada en píxeles  
Usa los modelos mencionados arriba (Diffusion, GANs, etc.)  

- Refinamiento:  
Mejora de resolución (super-resolución)  
Ajuste de detalles  
Filtrado de contenido inapropiado

**Modelos Principales**  
1. Diffusion Models (Modelos de Difusión)  
Proceso de "ruido": Comienzan con una imagen de puro ruido aleatorio  
Desruido guiado: Eliminan progresivamente el ruido en múltiples pasos  
Guía textual: En cada paso, el texto dirige cómo debe "evolucionar" la imagen  
Analogía: Como esculpir una estatua quitando material sobrante según un plano (el texto)

2. Generative Adversarial Networks (GANs)  
Dos redes neuronales compitiendo:  
Generador: Crea imágenes a partir de texto  
Discriminador: Juzga si las imágenes son reales o generadas  
Entrenamiento adversarial: Mejora continua mediante competición  
Resultado: El generador se vuelve experto en engañar al discriminador  

3. Autoregressive Models  
Tokenización de imagen: Tratan la imagen como una secuencia de "tokens" visuales  
Predicción secuencial: Generan la imagen token por token (como escribir una frase)  
Condicionamiento textual: Cada token se predice basándose en el texto y tokens anteriores